In [76]:
import pandas as pd
import DB as db
import numpy as np

In [83]:
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2022-12-11.zip"
log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-20.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-19.zip"
# log_summary_file_name = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Python-data\PortalLogs\summary\log summary-2023-04-18.zip"

logdf= pd.read_csv(log_summary_file_name, low_memory=False).query("token == 'confirmLandReservation True'")\
    .drop(columns=['node', 'task_id','project_id', 'error_line'])
# logdf.head()

In [100]:
land_app_fn = r"C:\Users\yahia\Downloads\land_Application.zip"
dbdf= pd.read_csv(land_app_fn, low_memory=False, dtype={'applicant_national_id':str})\
    # .query("project_id == 86 and land_application_status == 'W'")[['applicant_national_id', 'check_sum', 'land_id', 'project_id']]

In [103]:
dbdf.loc[dbdf.project_id == 86][['land_application_id', 'land_application_status']].groupby('land_application_status').count()

,land_application_id
land_application_status,
F,893
N,4586
P,9095
V,40
W,3343


In [ ]:
df = (dbdf.merge(logdf, left_on='applicant_national_id', right_on='NID')
      .assign(match = lambda x:(x.NA1 == x.check_sum))
    #   .fillna('land_id', axis=1)
    #   .assign(land_id = lambda x: x.astype(np.int64) )
    )
df.info()

In [80]:
df.match.value_counts()

match
True     410
False      1
Name: count, dtype: int64

In [95]:
df.query("match == False").assign(land_id = lambda x: x.land_id.astype(np.int64))

,applicant_national_id,check_sum,land_id,project_id,log_date,line_no,NID,log_type,country,IP_address,...,Region,District,Sub_District,Land_No,land_size,excellence_ratio,checksum,NA1,NA2,match
32,26312132702012,017B780E3DC6A2E24581C9763DE7CE9BEE894EEC745246...,80312,86,2023-04-20 10:16:43,3834604,26312132702012,ERROR,AE,217.165.114.211,...,قنا الجديدة,منطقة 600 فدان التوسعات الشرقية,-,الثانية,498.0,209.0,1.06,015975F7E08B1CA5FB85BE63226E492A48BDF0168BC5AE...,NaN,False
